#Government Scheme Finder

###Use cases: - A student searches for educational scholarships.

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 2.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import groq

In [ ]:
# Load the dataset
schemes_df = pd.read_csv('/content/government_schemes_csv.csv')

schemes_df.head()

,Scheme Name,Eligibility Criteria,Benefits,Application Process,Income Level,Age,Education,Gender
0,Sukanya Samriddhi Yojana,The account must be opened before age 10 by pa...,Sukanya Samriddhi Yojana offers tax exemptions...,Offline Application,150000,Below 10,School,Girl
1,National Fellowship for OBC Students,Must be an OBC member and enrolled in a full-t...,The scheme is open to OBC candidates pursuing ...,Online Application,600000,22-30,Ph.D.,Both
2,National overseas Scholarship,A scholarship requires at least 60% marks in t...,The National Overseas Scholarship covers tuiti...,Online Application,800000,upto 35,All,Both
3,Sarva Shiksha Abhiyan,Students aged 6 to 14 years are eligible to ap...,"Elementary education is free and compulsory, e...",Online Application,No income limit,6-14yrs,School,Both
4,Inclusive Education for the Disabled at Second...,The child must be in classes IX-XII at a gover...,"Inclusive education promotes social inclusion,...",Online Application,No income limit,6-18yrs,School,Both


In [ ]:

# Define a function to preprocess the data
def preprocess_data(schemes_df):
    # Convert the schemes data to lowercase
    schemes_df['scheme_name'] = schemes_df['Scheme Name'].str.lower()
    schemes_df['eligibility_criteria'] = schemes_df['Eligibility Criteria'].str.lower()
    schemes_df['age'] = schemes_df['Age'].astype(str).str.lower()
    schemes_df['gender'] = schemes_df['Gender'].str.lower()
    schemes_df['income'] = schemes_df['Income Level'].astype(str).str.lower()
    schemes_df['education'] = schemes_df['Education'].str.lower()

    return schemes_df

# Preprocess the data
schemes_df = preprocess_data(schemes_df)


In [ ]:
# Define a function to find eligible schemes
def find_eligible_schemes(user_input, schemes_df):
# Use TF-IDF to convert the user input and eligibility criteria to vectors
    vectorizer = TfidfVectorizer()

    # Fit the vectorizer to both user input and eligibility criteria
    all_text = [user_input] + schemes_df['eligibility_criteria'].tolist()
    vectorizer.fit(all_text)

    # Transform user input and eligibility criteria into vectors
    user_input_vector = vectorizer.transform([user_input])
    eligibility_criteria_vectors = vectorizer.transform(schemes_df['eligibility_criteria'])

    # Use cosine similarity to find the most similar schemes
    similarities = cosine_similarity(user_input_vector, eligibility_criteria_vectors)

    # Get the index of the most similar scheme
    most_similar_index = np.argmax(similarities)

    # Get the most similar scheme
    most_similar_scheme = schemes_df.iloc[[most_similar_index]]

    # Filter the schemes based on the user's demographic information
    # ... (Existing filtering logic) ...

    return most_similar_scheme # Return the most similar scheme

    # Filter the schemes based on the user's demographic information
    eligible_schemes = most_similar_schemes[most_similar_schemes['eligibility_criteria'].str.contains(schemes_df['Age'])]
    eligible_schemes = eligible_schemes[eligible_schemes['eligibility_criteria'].str.contains(schemes_df['Gender'])]
    eligible_schemes = eligible_schemes[eligible_schemes['eligibility_criteria'].str.contains(schemes_df['Income Level'])]
    eligible_schemes = eligible_schemes[eligible_schemes['eligibility_criteria'].str.contains(schemes_df['Education'])]

    return eligible_schemes




In [ ]:
#from transformers import pipeline # Not needed in this context
# Define a function to generate output using LLM
from groq import Groq
def generate_output(eligible_schemes, api_key):
    # Use Groq API to generate output
    client = Groq(api_key=api_key)

    # Use client.chat.completions.create() to interact with Groq models
    prompt = f"Generate a description for the following government schemes: {eligible_schemes['scheme_name'].tolist()}"
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ],
        model="llama3-8b-8192",  # or other appropriate model
    )
    output = response.choices[0].message.content  # Extract the generated text from the response

    return output


# Test the application

In [ ]:

#I am a student looking for educational scholarships."
user_input = "What initiatives or financial support options exist for disabled students to access higher education?"
eligible_schemes = find_eligible_schemes(user_input, schemes_df)
api_key = "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"
output = generate_output(eligible_schemes, api_key)

print(output)

Here is a possible description for the government scheme:

**Title:** Scholarship Scheme for Scheduled Caste Students for Abroad Studies

**Objective:** To provide financial assistance to Scheduled Caste students pursuing higher education abroad, with the goal of promoting educational excellence and social mobility among the marginalized community.

**Eligibility Criteria:**

* The applicant must be a Scheduled Caste student as per the Government of India's rules and notifications.
* The applicant must be selected for admission to a reputed foreign university or institution for a full-time undergraduate or postgraduate course.
* The applicant must not have any outstanding loan or scholarship provided by the Government of India.
* The applicant must be a citizen of India.

**Benefits:**

* The scholarship will cover tuition fees, living expenses, and other educational expenses incurred while pursuing higher education abroad.
* The scholarship amount will be determined based on the durat

In [ ]:
user_input = "What government programs or initiatives are available to support children who lost their parents to COVID-19 with their school expenses and education?"
eligible_schemes = find_eligible_schemes(user_input, schemes_df)
api_key = "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"
output = generate_output(eligible_schemes, api_key)

print(output)

Here's a description for the "Scholarship for PM Cares Children" government scheme:

**Scheme Name:** Scholarship for PM Cares Children

**Objective:** The Scholarship for PM Cares Children scheme is a flagship initiative of the Government of India to support the higher education of children who have lost one or both parents due to COVID-19. The scheme aims to provide financial assistance to these students, ensuring that they have access to quality education and a stable future.

**Eligibility Criteria:**

* The applicant must be a bonafide citizen of India.
* The applicant must have lost one or both parents due to COVID-19.
* The applicant must be pursuing higher education (Class 12 onwards) in a recognized institution.
* The applicant must have a minimum of 60% aggregate marks in the previous academic year.

**Scholarship Details:**

* The scholarship is open to students pursuing undergraduate and postgraduate courses in fields such as medicine, engineering, science, arts, and commer